<a href="https://colab.research.google.com/github/IBREEZZ/Code_Academy_Makeen2/blob/main/MYSQL(in_python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pymysql pandas

In [ ]:
# Step 1: Install MySQL Server
!apt-get update
!apt-get install -y mysql-server

# Step 2: Start the MySQL service
!service mysql start

# Step 3: Set up a sample MySQL database and user
!mysql -e "CREATE DATABASE colab_db;"  # Create a new database named 'colab_db'
!mysql -e "CREATE USER 'colab_user'@'localhost' IDENTIFIED BY 'password';"  # Create a new user
!mysql -e "GRANT ALL PRIVILEGES ON colab_db.* TO 'colab_user'@'localhost';"  # Grant all privileges to the user
!mysql -e "FLUSH PRIVILEGES;"  # Refresh privileges


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
# Step 4: Python code to connect to the MySQL server
import pymysql

try:
    # Connect to the MySQL server running in Colab
    connection = pymysql.connect(
        host='127.0.0.1',
        port=3306,
        user='colab_user',  # User created earlier
        password='password',  # Password created earlier
        database='colab_db'  # Database created earlier
    )
    print("Connected to the database!")

    # Step 5: Create a cursor to execute queries
    cursor = connection.cursor()

    # Example: Create a table
    cursor.execute("CREATE TABLE IF NOT EXISTS users (id INT, name VARCHAR(255))")
    print("Table created or already exists.")

    # Create products table
    cursor.execute("CREATE TABLE IF NOT EXISTS products (product_id INT PRIMARY KEY,product_name VARCHAR(255),price DECIMAL(10, 2));")

    # Create categories table
    cursor.execute("CREATE TABLE IF NOT EXISTS categories (category_id INT PRIMARY KEY,category_name VARCHAR(255));")

    # Create product_categories table
    cursor.execute("CREATE TABLE IF NOT EXISTS product_categories (product_id INT,category_id INT,PRIMARY KEY (product_id, category_id),FOREIGN KEY (product_id) REFERENCES products(product_id),FOREIGN KEY (category_id) REFERENCES categories(category_id));")

    print("products,categories,product_categories  inserted  !")
    # Example: Insert data into the table
    cursor.execute("INSERT INTO users (id, name) VALUES (1, 'John Doe')")
    cursor.execute("INSERT INTO users (id, name) VALUES (2, 'alica')")
    connection.commit()
    print("Data inserted!")

    # Example: Query data from the table
    cursor.execute("SELECT * FROM users")
    results = cursor.fetchall()
    print("Query Results:", results)

except pymysql.MySQLError as e:
    print("Error:", e)

finally:
    if connection:
        connection.close()
        print("Connection closed.")

Connected to the database!
Table created or already exists.
products,categories,product_categories  inserted  !
Data inserted!
Query Results: ((1, 'John Doe'), (1, 'John Doe'), (2, 'alica'), (1, 'John Doe'), (2, 'alica'), (1, 'John Doe'), (2, 'alica'), (1, 'John Doe'), (2, 'alica'), (1, 'John Doe'), (2, 'alica'))
Connection closed.


# **Exercise**

In [1]:
import sqlite3

# Connect to the database (or create it)
conn = sqlite3.connect('shop.db')
cursor = conn.cursor()


# **1**

In [3]:
# Create 'products' table
cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    price DECIMAL(10, 2)
)
""")

# Create 'categories' table
cursor.execute("""
CREATE TABLE IF NOT EXISTS categories (
    category_id INTEGER PRIMARY KEY,
    category_name TEXT NOT NULL
)
""")

# Create 'product_categories' (many-to-many relationship)
cursor.execute("""
CREATE TABLE IF NOT EXISTS product_categories (
    product_id INTEGER,
    category_id INTEGER,
    FOREIGN KEY (product_id) REFERENCES products(product_id) ON DELETE CASCADE,
    FOREIGN KEY (category_id) REFERENCES categories(category_id) ON DELETE CASCADE
)
""")

conn.commit()
print("tables created")


tables created


# **2**

In [4]:
# Insert products
products = [
    (1, 'Laptop', 1200.00),
    (2, 'Smartphone', 800.00),
    (3, 'Tablet', 300.00),
    (4, 'Headphones', 150.00),
    (5, 'Smartwatch', 200.00)
]
cursor.executemany("INSERT INTO products VALUES (?, ?, ?)", products)

# Insert categories
categories = [
    (1, 'Electronics'),
    (2, 'Wearables'),
    (3, 'Audio')
]
cursor.executemany("INSERT INTO categories VALUES (?, ?)", categories)

# Associate products with categories
product_categories = [
    (1, 1),  # Laptop -> Electronics
    (2, 1),  # Smartphone -> Electronics
    (3, 1),  # Tablet -> Electronics
    (4, 3),  # Headphones -> Audio
    (5, 2),  # Smartwatch -> Wearables
    (5, 1)   # Smartwatch -> Electronics
]
cursor.executemany("INSERT INTO product_categories VALUES (?, ?)", product_categories)

conn.commit()


# **3**

In [5]:
cursor.execute("SELECT product_name, price FROM products")
print(cursor.fetchall())


[('Laptop', 1200), ('Smartphone', 800), ('Tablet', 300), ('Headphones', 150), ('Smartwatch', 200)]


In [6]:
cursor.execute("""
SELECT c.category_name, p.product_name
FROM categories c
LEFT JOIN product_categories pc ON c.category_id = pc.category_id
LEFT JOIN products p ON pc.product_id = p.product_id
""")
print(cursor.fetchall())


[('Electronics', 'Laptop'), ('Electronics', 'Smartphone'), ('Electronics', 'Tablet'), ('Electronics', 'Smartwatch'), ('Wearables', 'Smartwatch'), ('Audio', 'Headphones')]


In [7]:
cursor.execute("""
SELECT c.category_name, p.product_name, MAX(p.price)
FROM categories c
JOIN product_categories pc ON c.category_id = pc.category_id
JOIN products p ON pc.product_id = p.product_id
GROUP BY c.category_id
""")
print(cursor.fetchall())


[('Electronics', 'Laptop', 1200), ('Wearables', 'Smartwatch', 200), ('Audio', 'Headphones', 150)]


In [8]:
cursor.execute("""
SELECT c.category_name, COUNT(pc.product_id) AS total_products
FROM categories c
LEFT JOIN product_categories pc ON c.category_id = pc.category_id
GROUP BY c.category_id
""")
print(cursor.fetchall())


[('Electronics', 4), ('Wearables', 1), ('Audio', 1)]


In [9]:
cursor.execute("""
SELECT c.category_name
FROM categories c
LEFT JOIN product_categories pc ON c.category_id = pc.category_id
WHERE pc.product_id IS NULL
""")
print(cursor.fetchall())


[]


# **4**

In [10]:
cursor.execute("UPDATE products SET price = 1300.00 WHERE product_id = 1")
conn.commit()


In [11]:
cursor.execute("DELETE FROM products WHERE product_id = 3")
conn.commit()


# **5**

In [12]:
cursor.execute("""
SELECT product_name, price
FROM products
ORDER BY price DESC
LIMIT 3
""")
print(cursor.fetchall())


[('Laptop', 1300), ('Smartphone', 800), ('Smartwatch', 200)]


In [13]:
cursor.execute("""
SELECT product_name, price
FROM products
WHERE price > (SELECT AVG(price) FROM products)
""")
print(cursor.fetchall())


[('Laptop', 1300), ('Smartphone', 800)]


In [14]:
def add_product_with_categories(product_id, name, price, category_ids):
    cursor.execute("INSERT INTO products VALUES (?, ?, ?)", (product_id, name, price))
    for category_id in category_ids:
        cursor.execute("INSERT INTO product_categories VALUES (?, ?)", (product_id, category_id))
    conn.commit()

# Example usage:
add_product_with_categories(6, 'Bluetooth Speaker', 100.00, [1, 3])


In [15]:
conn.close()
